### Step 1:

Working with windows from the entire image. The goal of this step is to segment the image and find the features of the image i.e. vacancies, adsorbed molecules. This is done by extracting the latent vectors from the bottleneck of an autoencoder for each window, clustering the latent vectors and reconstructing a cluster map with pixelwise clustering.

Based on this pixelwise clustering, a feature detector finds the features of the clustered image, stores their (y,x) position in the image. The convention for coordinates is (y,x) because image sizes are usually given in (height, width).

Then, we cut out windows of size (32,32) pixels around the position of the feature. These so called feature windows or `step2_windows` are then used in step 2.

### Step 2

Only working with feature windows extracted in step 1. The goal of this step is to cluster the feature windows i.e. classify the features into different classes using an unsupervised algorithm. This is done by retraining the autoencoder on the feature windows dataset to make it more tailored to extract features. The latent vectors are then clustered using a spectral clustering model. The result is an array of numbers, each number being a label.

Then we circle the features on the original image, each circle having a different colour, depending on the label of the feature.

In [2]:
# Define path where to find the module. This allows for a different path depending on where the code is running (my mac or the cluster)
import os
import sys

# Define candidate paths
module_path_list = [
    '/Users/steven/academic-iCloud/Python/modules',
    '/hpc/aklicka/Python/modules'
]

data_path_list = [
    '/Users/steven/Python-data',
    '/hpc/aklicka/Python-data/training-set-1'
]

# Resolve actual paths
module_path = next((p for p in module_path_list if os.path.exists(p)), None)
data_path = next((p for p in data_path_list if os.path.exists(p)), None)

# Check and report missing paths
if module_path is None:
    print("Error: Could not locate a valid module path.")
if data_path is None:
    print("Error: Could not locate a valid data path.")

if module_path is None or data_path is None:
    sys.exit(1)

# Print resolved paths
print(f"module_path = {module_path}")
print(f"data_path = {data_path}")

# Reduce TensorFlow verbosity
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


module_path = /hpc/aklicka/Python/modules
data_path = /hpc/aklicka/Python-data/training-set-1


In [3]:
# # Ensure modules are reloaded 
%load_ext autoreload
%autoreload 2

# Import standard modules
import numpy as np

import platform

from datetime import datetime

# Add custom module path to list
sys.path.append(module_path)

# Import custom module
import SRSML24.data_prep as dp
import SRSML24.model as m
import SRSML24.utils as ut

import tensorflow as tf
#from tensorflow.keras.optimizers.legacy import Adam 
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt

import pandas as pd
from IPython.display import display, Markdown

import skimage as ski
import skimage.morphology as morphology
import skimage
from skimage import morphology, measure

#import platform 

m.print_system_info()

start_time = dp.current_datetime()


Python version: 3.9.21 (main, Dec 11 2024, 16:24:11) 
[GCC 11.2.0]
TensorFlow version: 2.4.1
TensorFlow is built with CUDA: True
TensorFlow is built with ROCm: False

System: Linux 4.18.0-553.22.1.el8_10.x86_64 (x86_64)
Platform: Linux-4.18.0-553.22.1.el8_10.x86_64-x86_64-with-glibc2.28
Processor: x86_64

Number of GPUs available to TensorFlow: 1
GPU Device: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')

>>> Running with GPU available <<<  (Linux-4.18.0-553.22.1.el8_10.x86_64-x86_64-with-glibc2.28)

Current time 2025-06-25 12:00:19


### Programme variable setup

In [4]:
# Parameters for windows creation
# General
job_name = 'June_25_BIG_model'
verbose = False             # Set this True to print out more information

# MTRX preprocessing
flatten_method = 'poly_xy'
pixel_density = 15.0        # Convert all images to a constant pixel density
pixel_ratio = 0.7           # If an image has less than this % in the slow scan direction it is discarded
data_scaling = 1.e9         # Scale the z-height of the data

# Windowing
window_size = 32            # Window size for training/validation
window_pitch = 8            # Window pitch for training/validation

# Data saving options
save_windows = True         # Save the windows as numpy files
together = True             # Set this True to save image windows for a mtrx image as a single file rather than separate files.
save_jpg = False            # Save the full image as a jpg
collate = False             # Set this True to remove all subfolder directories and save all data in root data path
save_window_jpgs = False    # Save the windows as jpgs for inspection

# Parameters for training
model_name = 'unet_' + job_name
batch_size = 128
buffer_size = 12800 # shuffling
learning_rate = 1e-4
epochs = 5

# Parameters for clustering
cluster_model_name = model_name + '_kmeans'
cluster_batch_size = 5120 # This is the number of latent features in a batch for clustering. 
                          # Does not have to be the same as for training and probably should 
                          # be larger. 
cluster_buffer_size = cluster_batch_size * 5    # shuffling buffer
num_clusters=20                                 # Desired number of clusters (centroids) to form in the data.
max_iter=1000                                   # Maximum iterations allowed for each mini-batch to refine centroids.
reassignment_ratio=0.05                         # Fraction of clusters reassigned per step; lower values stabilize updates.

# Parameters for PREDICTIONS
predict_window_pitch = 2                        # Window pitch for prediction
predictions_batch_size = 2**15                  # Batch size for predictions


# DATA LIMITS FOR TESTING THE CODE
mtrx_train_data_limit = None                    # Number of MTRX files to process (training)
mtrx_test_data_limit = None                     # Number of MTRX files to process (validation)

train_data_limit = None                           # Limit the data used in the autoencoder training
test_data_limit = None                          # Limit the data used in the autoencoder training (validation)

# Step 2
cluster_model_spectral_name = model_name + "_spectral"

max_size_blob = 3000 #maximum pixel area of a blob to keep in the image
area_threshold = 50 #minimum pixel area of a blob to keep in the image
feature_size = 16 #radius of feature window taken from the centroid of the blob, actual window size is 2*feature_size

In [5]:

job_data_path = dp.create_new_data_path(data_path, job_name, include_date=False)

mtrx_train_path = os.path.join(data_path, 'mtrx/train')
mtrx_test_path = os.path.join(data_path, 'mtrx/test')
mtrx_predict_path = os.path.join(data_path, 'mtrx/predict')

# Step 1 paths
model_path = os.path.join(job_data_path,'model')
cluster_model_path = os.path.join(job_data_path,'cluster_model')

latent_features_path = os.path.join(job_data_path, 'latent_features')
predict_latent_features_path = os.path.join(job_data_path, 'latent_features_predictions')

windows_train_path = os.path.join(job_data_path, 'windows/train')
windows_test_path = os.path.join(job_data_path, 'windows/test')
windows_predict_path = os.path.join(job_data_path, 'windows/predict')

predictions_path = os.path.join(job_data_path, f'predictions')

# Step 2 paths
step2_model_path = os.path.join(job_data_path,'step2/model')
step2_cluster_model_path = os.path.join(job_data_path,'step2/cluster_model')

step2_latent_features_train_path = os.path.join(job_data_path, 'step2/latent_features/train')
step2_latent_features_test_path = os.path.join(job_data_path, 'step2/latent_features/test')
step2_latent_features_predict_path = os.path.join(job_data_path, 'step2/latent_features/predict')


step2_predict_latent_features_path = os.path.join(job_data_path, 'step2/latent_features_predictions')


step2_windows_train_path = os.path.join(job_data_path, 'step2/windows/train')
step2_windows_test_path = os.path.join(job_data_path, 'step2/windows/test')
step2_windows_predict_path = os.path.join(job_data_path, 'step2/windows/predict')

step2_predictions_path = os.path.join(job_data_path, f'step2/predictions')



### Process Matrix format data to windows for making predictions 

In [ ]:
dp.delete_data_folders(
    job_data_path, 
    subdirectories=['windows/predict','windows-jpg/predict','jpg/predict'],
    override=True)

In [ ]:
# Prediction data in MTRX format
mtrx_predict_file_list, _ = dp.list_files_by_extension(mtrx_predict_path,'Z_mtrx',verbose=False)

dp.process_mtrx_files(
    mtrx_predict_file_list,
    job_data_path, # save data path
    flatten_method = flatten_method, pixel_density = pixel_density, pixel_ratio = pixel_ratio,
    data_scaling = data_scaling, window_size = window_size, 
    window_pitch = predict_window_pitch,
    save_windows = save_windows,
    save_window_jpgs=save_window_jpgs,
    save_jpg = save_jpg,
    together = together,
    collate = collate,
    verbose = verbose
)

### Step 1: Making cluster image. Make predictions using the trained autoencoder and KMEANS models. Extract feature windows

In step 1, all windows are used. Latent vectors relate to windows extracted from the entire image.

In [ ]:
# Load the trained autoencoder
autoencoder_model = m.load_model(model_path, model_name=model_name)

# Load a previously saved cluster model from disk
cluster_model = m.load_cluster_model(cluster_model_path, model_name=cluster_model_name)

In [ ]:
step1_predict_files, step1_num_predict = dp.list_files_by_extension(windows_predict_path, 'npy')

# Get the corresponding image coordimages list file
step1_prediction_windows_coordinates_file_list , _ = dp.list_files_by_extension(windows_predict_path,'.txt',verbose=False)
step1_prediction_windows_coordinates_file_list = [
    name for name in step1_prediction_windows_coordinates_file_list 
    if "coordinates" in name
]


### The following takes about 10 mins for 32 images

In [ ]:
dp.delete_data_folders(job_data_path, subdirectories=["step2/windows/predict"], override=True)


# Extracting the feature windows from the prediction dataset 
m.extract_and_save_feature_windows(step1_predict_files, step1_prediction_windows_coordinates_file_list, autoencoder_model, cluster_model, 
                                 window_size=window_size, predictions_batch_size=predictions_batch_size, feature_size=feature_size, save_path=step2_windows_predict_path, verbose=True)



### Step 2

In [6]:
# Load the trained autoencoder
step2_autoencoder_model = m.load_model(step2_model_path, model_name=model_name)


# Load a previously saved cluster model from disk
step2_cluster_model_kmeans = m.load_cluster_model(step2_cluster_model_path, model_name=cluster_model_name)
step2_cluster_model_spectral = m.load_cluster_model(step2_cluster_model_path, model_name=cluster_model_spectral_name)


InternalError: CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory

In [ ]:
image_num = 29



# Step 2
windows, labels = m.predict_labels_from_feature_windows(step2_windows_predict_path, step2_latent_features_predict_path, job_data_path, step2_autoencoder_model, step2_cluster_model_spectral,
                                                        cluster_model_type="spectral", image_num = image_num, feature_size = 16, predictions_batch_size=2**15)

# Sort the labels numerically, while also sorting the corresponding feature windows
sorted_indices = np.argsort(labels)
sorted_labels = labels[sorted_indices]
sorted_feature_windows = windows[sorted_indices]
m.display_feature_windows_with_labels(sorted_feature_windows, sorted_labels, num_images=min(windows.shape[0],100))


In [ ]:

#get reconstructed image and cluster image
reconstructed_img, cluster_img = m.reconstruct_predict(step1_predict_files[image_num], step1_prediction_windows_coordinates_file_list[image_num], autoencoder_model, cluster_model, window_size, predictions_batch_size)

# Get the corresponding image coordimages list file
step2_prediction_windows_coordinates_file_list , _ = dp.list_files_by_extension(step2_windows_predict_path,'.txt',verbose=False)

# Get the coordinates for a specific image with index `image_num`
coordinates_data = np.loadtxt(step2_prediction_windows_coordinates_file_list[image_num], skiprows=1, dtype=float).astype(np.int64)

# Show the original image with all found features in different coloured squares
m.display_labels_on_image(reconstructed_img, coordinates_data, labels, label_num=[2,4], show_cluster_img=True, cluster_img=cluster_img)

m.display_labels_on_image(reconstructed_img, coordinates_data, labels, label_num=[], show_cluster_img=True, cluster_img=cluster_img)